In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pywt as pw
import wavelet_filter as filter
from ecg_gudb_database import GUDb
from qrs_detection import QRS_detection as detect
from hrv import HRV

In [ ]:
data = pd.read_excel("data.xlsx", sheet_name="0-sitting",
                     header=0, usecols="B:F")
data = data[0]
data = (data - min(data)) / (max(data) - min(data))
ecg = data.T.values.tolist()

In [ ]:

sample_rate = GUDb.fs
t = np.arange(len(ecg)) / float(sample_rate)
filtered = filter.wavelet_filtering(data)
plt.xlabel('Time')
plt.ylabel('ECG Signal')
plt.xlim(0, 3)
plt.plot(t, filtered)

In [ ]:
plt.xlabel('Time')
plt.ylabel('ECG Signal')
ecg = np.array(ecg)
t = np.arange(len(ecg)) / float(250)
ecg = (ecg - min(ecg)) / (max(ecg) - min(ecg))
plt.xlim(0, 3)
plt.plot(t, ecg)

In [ ]:
data += np.random.normal(0, 0.05, (data.shape))
t = np.arange(len(data)) / float(250)
plt.xlim(0, 3)
plt.plot(t, data)

In [ ]:
import wavelet_filter as filter
from ecg_gudb_database import GUDb
sample_rate = GUDb.fs
filtered = filter.wavelet_filtering(data)
plt.xlabel('Time')
plt.ylabel('ECG Signal')
plt.xlim(0, 3)
plt.plot(t, filtered)

In [ ]:
spot = detect(filtered, GUDb.fs, 160)
spot_t = []
spot_r = []
for i in spot:
    spot_t.append(t[i])
for i in spot:
    spot_r.append(ecg[i])
plt.xlim(0, 8)
plt.plot(spot_t, spot_r, 'x', color='r')
plt.plot(t, filtered)

In [ ]:
pw.swt_max_level(len(ecg))

In [ ]:
(cA4, cD4), (cA3, cD3), (cA2, cD2), (cA1, cD1) = pw.swt(ecg, wavelet="haar", level=4)

In [ ]:
plt.figure(figsize=(6, 7))
plt.subplots_adjust(hspace =0.5)
plt.subplot(411)
plt.xlim(0, 4)
plt.title("cD4")
plt.plot(t, cD4)

plt.subplot(412)
plt.xlim(0, 4)
plt.title("cD3")
plt.plot(t, cD3)

plt.subplot(413)
plt.xlim(0, 4)
plt.title("cD2")
plt.plot(t, cD2)

plt.subplot(414)
plt.xlim(0, 4)
plt.title("cD1")
plt.plot(t, cD1)

plt.suptitle("Detail Coefficents Use Haar Wavelet Base")


In [ ]:
avg = np.mean(cD3)
std = np.std(cD3)
sig_thres = [abs(i) if abs(i) > 2.0*std else 0 for i in cD3-avg]
plt.xlim(0, 2)
plt.plot(t, sig_thres)

In [ ]:
input = []
hrv_class = HRV(250)

In [ ]:
for i in range(20):
    data = pd.read_excel("data.xlsx", sheet_name=str(i) + "-sitting",
                         header=0, usecols="B:F")
    data = data[0]
    data = (data - min(data)) / (max(data) - min(data))
    filtered = filter.wavelet_filtering(data)
    spot = detect(filtered, GUDb.fs, 160)
    input.append(hrv_class.pNN50(spot))
    input.append(hrv_class.RMSSD(spot, normalise=True))
    input.append(hrv_class.SDNN(spot, normalise=True))

In [ ]:
f_input = np.array(input).reshape(40, 3)

In [ ]:
f_label = []
for i in range(20):
    f_label.append([1])
for i in range(20):
    f_label.append([0])

In [ ]:
f_label = np.array(f_label).reshape(40, 1)
f_label

In [ ]:
from bp_neural_network import BPNN
bp_instance = BPNN()
bp_instance.setup(3, 3, 1)
bp_instance.train(f_input, f_label, 20000, 0.2, 0.1)

In [ ]:
result = []
for i in f_input:
    result.append(1 if(bp_instance.predict(i)[0] > 0.5) else 0)

In [ ]:
sum = 0
for i in range(20):
    sum += f_label[i][0] ^ result[i]
print(sum)

In [ ]:
test = []
data = pd.read_excel("data.xlsx", sheet_name="20-sitting",
                     header=0, usecols="B:F")
data = data[0]
data = (data - min(data)) / (max(data) - min(data))
filtered = filter.wavelet_filtering(data)
spot = detect(filtered, GUDb.fs, 160)
test.append(hrv_class.pNN50(spot))
test.append(hrv_class.RMSSD(spot, normalise=True))
test.append(hrv_class.SDNN(spot, normalise=True))
test.append(hrv_class.fAnalysis(spot))

In [ ]:
sum = 0

In [ ]:
for i in range(5):
    test = []
    file_no = i + 20
    data = pd.read_excel("data.xlsx", sheet_name=str(file_no) + "-sitting",
                        header=0, usecols="B:F")
    data = data[0]
    data = (data - min(data)) / (max(data) - min(data))
    filtered = filter.wavelet_filtering(data)
    spot = detect(filtered, GUDb.fs, 160)
    test.append(hrv_class.pNN50(spot))
    test.append(hrv_class.RMSSD(spot, normalise=True))
    test.append(hrv_class.SDNN(spot, normalise=True))
    sum += 0 ^ (1 if (bp_instance.predict(test)[0]) > 0.5 else 0)


In [ ]:
print(str((10 - sum) / 10 * 100) + "%")